## **AI Data Analysis**

Moving Average Time Series Model
Vizualize data through 

Autoregressive model

### **DataFrame**
Grabs the training and testing dataset from google drive

In [1]:
import pandas as pd
training_url = 'https://drive.google.com/file/d/1uVq3dwf4CKwLd5U0Y8uMOjxkjjXmXfvO/view?usp=sharing'
testing_url='https://drive.google.com/file/d/1tELkS1DEUddnpKkguctOj6t8LvgJYDIz/view?usp=sharing'
training_file_id=training_url.split('/')[-2]
testing_file_id = testing_url.split('/')[-2]
teest_dwn_url='https://drive.google.com/uc?id=' + testing_file_id
train_dwn_url = 'https://drive.google.com/uc?id=' + training_file_id
training_dataset = pd.read_csv(train_dwn_url)
testing_dataset = pd.read_csv(train_dwn_url)
print(training_dataset.head())
print(testing_dataset.head())

   group_id          arrive_time          depart_time  Fleet_Number   
0    2375.0  2023-05-15 07:35:09  2023-05-15 07:35:49           934  \
1    1510.0  2023-05-15 07:35:50  2023-05-15 07:36:10           934   
2    2739.0  2023-05-15 07:36:11  2023-05-15 07:36:30           934   
3    1385.0  2023-05-15 07:36:32  2023-05-15 07:37:12           934   
4    1505.0  2023-05-15 07:37:54  2023-05-15 07:39:15           934   

   Timepoint  Time Difference Type Time      Group_Name  
0          1  0 days 00:03:26    depart  bayhi_to_oakes  
1          0  0 days 00:03:26       NaN  bayhi_to_oakes  
2          0  0 days 00:03:26       NaN  bayhi_to_oakes  
3          0  0 days 00:03:26       NaN  bayhi_to_oakes  
4          1  0 days 00:03:26    depart  bayhi_to_oakes  
